<a href="https://colab.research.google.com/github/uk-zash/OnlineOrbit/blob/main/Result_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import AutoTokenizer , AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")

if tokenizer.pad_token is None:
    # Set the EOS token as the pad token if it's not set
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
clm_data = []

for entry in data:
    clm_data.append({
        "input_text": entry['question'],
        "target_text": entry['answer']
    })


In [3]:
# Add a new padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))  # Resize model embeddings to account for the new pad token


Embedding(50258, 768)

In [4]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [6]:
from datasets import load_dataset

# Load your dataset (adjust the path as necessary)
dataset = load_dataset('json', data_files='/content/clm_data.json')


Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 399
    })
})

In [8]:
def preprocess_function(examples):
    tokenized_outputs = tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        return_overflowing_tokens=True,
        return_offsets_mapping=False,  # This can help reduce complexity
        padding=False
    )

    # Initialize lists to store flattened results
    input_ids = []
    attention_masks = []

    # Loop through each set of tokenized outputs
    for input_ids_batch, attention_mask_batch in zip(tokenized_outputs['input_ids'], tokenized_outputs['attention_mask']):
        input_ids.extend(input_ids_batch)     # Collect all input_ids from the batch
        attention_masks.extend(attention_mask_batch)  # Collect all attention masks from the batch

    # Return the flattened outputs for map function
    return {
        'input_ids': input_ids,
        'attention_mask': attention_masks
    }


In [9]:
from transformers import DataCollatorForLanguageModeling

# Initialize data collator with dynamic padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Set to False for causal language modeling
    pad_to_multiple_of=None  # Ensure dynamic padding (not fixed to a specific length)
)


In [10]:
# Tokenize the entire dataset
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["text"]  # Remove original columns to avoid length mismatch
)


Map:   0%|          | 0/399 [00:00<?, ? examples/s]

In [11]:
split_dataset = tokenized_dataset["train"].train_test_split(test_size = 0.1)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

In [12]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./result_llm",
    overwrite_output_dir=True,
    num_train_epochs=3,              # Number of epochs
    per_device_train_batch_size=4,   # Batch size per device
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=500,
    evaluation_strategy="steps",
    eval_steps=500,
    learning_rate=5e-5,              # Learning rate
    fp16=False                        # Use mixed precision if supported
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
from transformers import Trainer

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [14]:
trainer.train()

Step,Training Loss,Validation Loss
500,8.289000,nan
1000,6.835700,nan
1500,6.741300,nan
2000,6.610100,nan
2500,6.634800,nan
3000,6.687100,nan
3500,6.602000,nan
4000,6.607800,nan
4500,6.546500,nan
5000,6.564300,nan


TrainOutput(global_step=27057, training_loss=6.397821374719704, metrics={'train_runtime': 1457.6877, 'train_samples_per_second': 74.246, 'train_steps_per_second': 18.562, 'total_flos': 27616821018624.0, 'train_loss': 6.397821374719704, 'epoch': 3.0})

In [15]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: nan


In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

events.out.tfevents.1724904798.78287ca2ad8f.635.0:   0%|          | 0.00/31.8k [00:00<?, ?B/s]

events.out.tfevents.1724906267.78287ca2ad8f.635.1:   0%|          | 0.00/364 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ukzash1/result_llm/commit/0241fa9fe13907c4686a4568c63d5ee026e5cce5', commit_message='End of training', commit_description='', oid='0241fa9fe13907c4686a4568c63d5ee026e5cce5', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from transformers import pipeline

In [23]:
from transformers import AutoTokenizer

# Load the tokenizer from the base model
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Save the tokenizer files locally
tokenizer.save_pretrained("/content/tokenizer_files")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

('/content/tokenizer_files/tokenizer_config.json',
 '/content/tokenizer_files/special_tokens_map.json',
 '/content/tokenizer_files/vocab.json',
 '/content/tokenizer_files/merges.txt',
 '/content/tokenizer_files/added_tokens.json',
 '/content/tokenizer_files/tokenizer.json')

In [26]:
from huggingface_hub import HfApi

# Initialize the Hugging Face API
api = HfApi()

# Set your repository ID
repo_id = "ukzash1/result_llm"

# List of tokenizer files to upload
tokenizer_files = ["vocab.json", "merges.txt", "tokenizer.json", "tokenizer_config.json"]

# Upload each tokenizer file to the Hugging Face repository
for file_name in tokenizer_files:
    api.upload_file(
        path_or_fileobj=f"/content/tokenizer_files/{file_name}",  # Local path to the file
        path_in_repo=file_name,  # Path in the repository
        repo_id=repo_id,  # Your model repo ID
        repo_type="model"  # Type of repo (model)
    )

print("Tokenizer files uploaded successfully!")


Tokenizer files uploaded successfully!


In [27]:
from transformers import pipeline

# Load the model and tokenizer using the pipeline
chat = pipeline("text-generation", model="ukzash1/result_llm")

# Test the model by generating some text
output = chat("Hello, how are you?", max_length=50)
print(output)


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Hello, how are you? include usedWhat? environmental, is can crop with and health to used a the, to. products. products, farming practicesWhat, and plants with used in and° of and?., or of, cultivation also'}]


In [33]:


# Generate text with adjusted parameters
response = chat("What is the best condition for rice to grow?", max_length=100, temperature=0.7, top_p=0.9)
# print(output)



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [34]:
response[0]["generated_text"]

'What is the best condition for rice to grow? and, the, andWhat are of cultivation,,-,, soil,, with are., and?? and the.. in is-, to. and the and is and is and in a. used and? The include. and, the- plant, and and are,, and. a the in? plant.. such a are,. and is. and also to ofWhat and. with include growth,,'

In [35]:
questions = [
    "What are the best conditions for growing rice?",
    "How does soil type affect rice cultivation?",
    "What are common pests in rice farming?"
]

for question in questions:
    output = chat(question, max_length=150, temperature=0.7, top_p=0.9)
    print(f"Question: {question}\nAnswer: {output[0]['generated_text']}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: What are the best conditions for growing rice?
Answer: What are the best conditions for growing rice?.. are and and. and? soilWhat- and, soil. to The.. for. is used in and,, for., are, pest used in soil. and, water are, practices is such of in used growth, health and in for of and? soil,, is in,. soil, the, practices soil?What, as,, the,, growth. and the in growth used for and,,,.,, also and to are?,, the and? are are health,-, can a, can the, and to, are. and as the the,.-. is, growth such and in and,. and.,



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Question: How does soil type affect rice cultivation?
Answer: How does soil type affect rice cultivation? and for..,What are,., growth to used. such., and of for. a are for in, and to and farming..,. to the in to pest., and plant. are and. and also is. as health areWhat. health and,.. are.? are, to, in- and in are a. used are soil and are,.,.? and to water used and of to health is, the plant. and and and practices are and,? the growth the, cultivation is, water the plant, The and? are and soil andWhat and management,., in and and,, can? cultivation in for are cultivation

Question: What are common pests in rice farming?
Answer: What are common pests in rice farming?., and can are a is are. of, used the is, and and.... are the are. and and growth. of growth and used are of, and is. the. to of are, the and growth.. include?. practices.. soil and,. in and are?, isWhat a toWhat,. as the.,., a of and, are pest such growth a and, and the? to used for and to soil.? water is and farming growth